# Loading WHO data with dlt

In [ ]:
import dlt

In [ ]:
import dlt
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.paginators import PageNumberPaginator

maximum_page = 3000


# data_sets= {"dimension": }

# Define the API resource for WHO data
@dlt.resource(name="who")   # <--- The name of the resource (will be used as the table name)
def air_data():
    client = RESTClient(
        base_url="https://ghoapi.azureedge.net/api/",
        paginator=PageNumberPaginator(
            base_page=1,
            total_path=None,
            maximum_page=1
        )
    )

    for page in client.paginate("AIR_6", params={"size": 100}):    # <--- API endpoint for retrieving taxi ride data
        yield page   # <--- yield data to manage memory


# define new dlt pipeline
pipeline = dlt.pipeline(pipeline_name="who_air", destination="duckdb", dataset_name="air_6")

# run the pipeline with the new resource
load_info = pipeline.run(air_data, write_disposition="replace")

In [ ]:
# Start a connection to your database using native `duckdb` connection and look what tables were generated:

import duckdb

# A database '<pipeline_name>.duckdb' was created in working directory so just connect to it

# Connect to the DuckDB database
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# Set search path to the dataset
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")

# Describe the dataset
conn.sql("DESCRIBE").df()

In [ ]:
conn.sql("select * from who_air.air_6.air").df()